In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Initialize Mediapipe Hands Model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.9)

# Data Directory
data_dir = 'ASL_Alphabet_Dataset/asl_alphabet_train'

# Data Storage
data = []
labels = []

# Process Images
print("Processing images...")
for label in sorted(os.listdir(data_dir)):
    if label == '.DS_Store':  # Ignore system files
        continue

    label_path = os.path.join(data_dir, label)
    for img_file in os.listdir(label_path):
        img_path = os.path.join(label_path, img_file)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Extract Hand Landmarks
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmark_data = []
                
                # ✅ Store only X and Y (Ignore Z)
                for point in hand_landmarks.landmark:
                    landmark_data.append(point.x)  # X-coordinate
                    landmark_data.append(point.y)  # Y-coordinate

                data.append(landmark_data)
                labels.append(label)

    print(f"Processed Class: {label}")

# Convert to Numpy Arrays
data = np.array(data)
labels = np.array(labels)

# Encode Labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(
    data, labels_categorical, test_size=0.15, random_state=22, shuffle=True
)

# Save Label Encoder
with open("label_encoder.pickle", "wb") as f:
    pickle.dump(label_encoder, f)

# CNN Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')  # Output Layer
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
print("Training Model...")
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Save Model
model.save("asl_cnn_2D_model.h5")

# Print Final Accuracy
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Save Model
with open("model2D.p", "wb") as f:
    pickle.dump({'model': model}, f)

print("Model training complete and saved successfully!")


Processing images...
Processed Class: A
Processed Class: B
Processed Class: C
Processed Class: D
Processed Class: E
Processed Class: F
Processed Class: G
Processed Class: H
Processed Class: I
Processed Class: J
Processed Class: K
Processed Class: L
Processed Class: M
Processed Class: N
Processed Class: O
Processed Class: P
Processed Class: Q
Processed Class: R
Processed Class: S
Processed Class: T
Processed Class: U
Processed Class: V
Processed Class: W
Processed Class: X
Processed Class: Y
Processed Class: Z
Processed Class: del
Processed Class: nothing
Processed Class: space


C:\Users\shree\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training Model...
Epoch 1/50
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.4057 - loss: 2.0133 - val_accuracy: 0.9173 - val_loss: 0.4088
Epoch 2/50
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8391 - loss: 0.5483 - val_accuracy: 0.9512 - val_loss: 0.2366
Epoch 3/50
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8927 - loss: 0.3755 - val_accuracy: 0.9608 - val_loss: 0.1744
Epoch 4/50
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9107 - loss: 0.3145 - val_accuracy: 0.9614 - val_loss: 0.1561
Epoch 5/50
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9235 - loss: 0.2716 - val_accuracy: 0.9673 - val_loss: 0.1380
Epoch 6/50
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9259 - loss: 0.2595 - val_accuracy: 0.9654 - val_loss: 0.1341
Epoch 7/50
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9337 - loss: 0.2286 - val_accuracy: 0.9713 - val_loss: 0.1164
Epoch 8/50
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9386

462/462 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9861 - loss: 0.0523
Test Accuracy: 0.9861
Model training complete and saved successfully!
